In [1]:
import pandas as pd
import numpy as np

# new version

In [2]:
hazards_adrien = pd.read_csv("data/disasters/fa_guessed_from_GAR_events_and_PAGER_shaved_sept16.csv")

In [46]:
hazards_adrien.sample(10)

,country,hazard,rp,fa
4293,Sri Lanka,flood,4000.0,0.065948
1428,El Salvador,earthquake,50.0,0.039048
2493,Japan,earthquake,10.0,0.006942
4794,Uganda,flood,1000.0,0.060939
4771,Uganda,earthquake,3.0,0.000751
3107,Mexico,surge,0.1,0.000000
2384,Israel,tsunami,5000.0,0.015323
4219,South Africa,flood,250.0,0.013625
4673,Togo,flood,1000.0,0.073601
2480,Jamaica,wind,3.0,0.000096


In [25]:
hazards_adrien.hazard.unique()

array(['earthquake', 'flood', 'tsunami', 'surge', 'wind'], dtype=object)

In [3]:
hazards_adrien['iso3'] = hazards_adrien.country.replace(pd.read_csv("data/disasters/names.csv", index_col='country').iso3)

In [4]:
drought = pd.read_excel("data/disasters/droughts_005_025_100.xlsx","Sheet1")
keep = drought[['iso_code','RP005_pop_aff_pct','RP025_pop_aff_pct','RP100_pop_aff_pct']].copy()
keep = keep.rename(columns={'iso_code':'iso3','RP005_pop_aff_pct':5,\
                            'RP025_pop_aff_pct':25,'RP100_pop_aff_pct':100})

In [5]:
hop = keep.set_index('iso3')
hop.columns.name = 'rp'
hop=hop.stack()
hop.name="fa"
hop = hop.reset_index()
hop['hazard'] = 'drought'
hop.sample(3)

,iso3,rp,fa,hazard
36,CHL,5,0.000001,drought
226,URY,25,0.000473,drought
217,TKM,100,0.064512,drought


In [6]:
hop['country'] = hop.iso3.replace(pd.read_csv("data/disasters/names.csv", index_col='iso3').country)

In [7]:
hazards_adrien = hazards_adrien.append(hop,ignore_index=True)

In [8]:
deltares_floods = pd.read_csv("data/disasters/deltares_floods_rp.csv", index_col="country")
deltares_floods.columns.name="rp"
foo=pd.DataFrame(deltares_floods.stack(), columns=["fa"]).reset_index()
foo['iso3'] = foo.country.replace(pd.read_csv("data/disasters/names.csv", index_col='country').iso3)
foo['hazard'] = 'floodglofris'

foo.rp = foo.rp.astype(float)

In [9]:
hazards_adrien = hazards_adrien.append(foo,ignore_index=True)

In [33]:
hazards_adrien.reset_index().to_csv("data/disasters/hazard_ratios_sept2016_plus_drought.csv",index=False,encoding='utf8')

In [20]:
v = pd.read_csv("data/disasters/v_pr_fromPAGER_shaved_GARevents_sept16.csv")

In [21]:
v['iso3'] = v.country.replace(pd.read_csv("data/disasters/names.csv", index_col='country').iso3)

In [18]:
r = pd.read_csv("data/disasters/results_august2016.csv")

In [19]:
r['iso3'] = r.country.replace(pd.read_csv("data/disasters/names.csv", index_col='country').iso3)

In [144]:
r.columns

Index(['country', 'gdp_pc_pp', 'pop', 'axhealth', 'urbanization_rate',
       'prepare_scaleup', 'borrow_abi', 'avg_prod_k', 'T_rebuild_K', 'pi',
       'income_elast', 'rho', 'shareable', 'max_increased_spending',
       'protection', 'fa_glofris', 'tau_tax', 'macro_multiplier', 'dK',
       'dKtot', 'delta_W', 'delta_W_tot', 'average_aid_cost_pc', 'axfin', 'c',
       'dc', 'dc_npv_post', 'dc_npv_pre', 'dk', 'dw', 'fa', 'gamma_SP',
       'help_fee', 'help_received', 'k', 'n', 'shew', 'social', 'v', 'v_shew',
       'dWpc_currency', 'dWtot_currency', 'risk', 'resilience',
       'risk_to_assets', 'iso3'],
      dtype='object')

In [145]:
v['avg_prod_k'] = v.iso3.replace(r.set_index('iso3')['avg_prod_k'])
v['macro_multiplier'] = v.iso3.replace(r.set_index('iso3')['macro_multiplier'])

In [36]:
v.reset_index().to_csv("data/disasters/cat_info_sept2016.csv",index=False)

In [22]:
v.reset_index().to_csv("C:\\Users\\julierozenberg\\Documents\\GitHub\\poverty_climate_model\\data\\disasters\\vulnerabilities.csv",index=False)

In [38]:
v.sample(3)

,country,vp,vr,v,iso3
93,High income,NaN,NaN,NaN,High income
105,Isle of Man,0.3,0.268852,NaN,IMN
189,Saudi Arabia,0.3,0.300000,NaN,SAU


In [42]:
hazards_adrien[hazards_adrien.hazard=='tsunami'].describe()

,fa,rp
count,7.360000e+02,736.000000
mean,1.197505e-02,4098.648098
std,5.051861e-02,8162.665175
min,0.000000e+00,0.500000
25%,9.060000e-08,250.000000
50%,1.658985e-04,1500.000000
75%,3.563624e-03,4000.000000
max,8.346488e-01,30000.000000


In [43]:
v.describe()

,vp,vr,v
count,206.000000,206.000000,140.000000
mean,0.553364,0.360989,0.391645
std,0.174005,0.176674,0.180204
min,0.105000,0.100000,0.100270
25%,0.394870,0.239500,0.247753
50%,0.684783,0.300000,0.368468
75%,0.700000,0.464037,0.545951
max,0.700000,0.697399,0.697535


# for climate change model

In [10]:
def average_over_rp(df):        
    ###AGGREGATION OF THE OUTPUTS OVER RETURN PERIODS
    
    #does nothing if df does not contain data on return periods
    try:
        if "rp" not in df.index.names:
            print("rp was not in df")
            return df
    except(TypeError):
        pass
    
    #just drops rp index if df contains default_rp
    # if default_rp in df.index.get_level_values("rp"):
        # print("default_rp detected, droping rp")
        # return df.reset_index("rp",drop=True)
        
    df=df.copy().reset_index("rp")

    #computes probability of each return period
    return_periods=np.unique(df["rp"].dropna())

    proba = pd.Series(np.diff(np.append(1/return_periods,0)[::-1])[::-1],index=return_periods) #removes 0 from the rps

    #matches return periods and their probability
    proba_serie=df["rp"].replace(proba)


    #handles cases with multi index and single index (works around pandas limitation)
    idxlevels = list(range(df.index.nlevels))
    if idxlevels==[0]:
        idxlevels =0
        
    #average weighted by proba
    averaged = df.mul(proba_serie,axis=0).sum(level=idxlevels) # obsolete .div(proba_serie.sum(level=idxlevels),axis=0)
    
    return averaged.drop("rp",axis=1)

In [11]:
hop = pd.DataFrame(hazards_adrien.set_index(['iso3','hazard','rp'])['fa'],columns=['fa'])
hop.columns.name="var"

In [12]:
tak=hop.stack().unstack("hazard").unstack("rp").copy()

In [15]:
paf = pd.DataFrame()
for hazard in ['earthquake', 'surge','tsunami', 'wind']:
    temp = tak[hazard].copy()
    
    temp2 = average_over_rp(temp.unstack().stack("rp"))
    temp2['hazard_name'] = hazard
    paf = paf.append(temp2)

In [16]:
for hazard_name in ['drought']:

    temp = tak[hazard_name].copy()

    b = temp[25]-(temp[25]-temp[5])/(25-5)*25
    temp[2]=b+(temp[25]-temp[5])/(25-5)*2
    temp[2]=temp[2].clip(lower=0)

    temp2 = average_over_rp(temp.unstack().stack("rp"))
    temp2['hazard_name'] = 'drought'
    
    paf = paf.append(temp2)

In [17]:
for hazard_name in ['floodglofris']:

    temp = tak[hazard_name].copy()

    b = temp[10]-(temp[10]-temp[5])/(10-5)*10
    temp[2]=b+(temp[10]-temp[5])/(10-5)*2
    temp[2]=temp[2].clip(lower=0)

    temp2 = average_over_rp(temp.unstack().stack("rp"))
    temp2['hazard_name'] = 'floodglofris'
    
    paf = paf.append(temp2)

In [18]:
paf.reset_index().to_csv("C:\\Users\\julierozenberg\\Documents\\GitHub\\poverty_climate_model\\data\\disasters\\av_fa_for_cc_model.csv",index=False)

In [19]:
paf.fa.describe()

count    585.000000
mean       0.007552
std        0.014751
min        0.000000
25%        0.000045
50%        0.000951
75%        0.009357
max        0.150831
Name: fa, dtype: float64

# old version

In [53]:
hazards = pd.read_csv("data/disasters/fa_guessed_from_GAR_and_PAGER_GLOFRIS.csv")

In [60]:
hazards.fa.describe()

count    6448.000000
mean        0.043559
std         0.106339
min         0.000000
25%         0.000000
50%         0.002525
75%         0.043863
max         1.536068
Name: fa, dtype: float64

In [61]:
hazards.ix[hazards.fa>1,:]

,countryname,fa,hazard,iso3,rp
111,Armenia,1.067490,earthquake,ARM,1500
1286,Dominican Republic,1.046351,wind,DOM,1000
1287,Dominican Republic,1.096707,wind,DOM,1500
2209,Jamaica,1.169987,wind,JAM,500
2210,Jamaica,1.315944,wind,JAM,1000
2211,Jamaica,1.355704,wind,JAM,1500
6224,Bahrain,1.340386,drought,BHR,100
6262,Dominica,1.183643,drought,DMA,100
6272,Western Sahara,1.160298,drought,ESH,100
6281,Falkland Islands (Malvinas),1.536068,drought,FLK,100


In [3]:
drought = pd.read_excel("data/disasters/droughts_005_025_100.xlsx","Sheet1")

In [32]:
keep = drought[['iso_code','RP005_pop_aff_pct','RP025_pop_aff_pct','RP100_pop_aff_pct']].copy()

In [33]:
keep = keep.rename(columns={'iso_code':'iso3','RP005_pop_aff_pct':5,\
                            'RP025_pop_aff_pct':25,'RP100_pop_aff_pct':100})

In [34]:
hop = keep.set_index('iso3')
hop.columns.name = 'rp'
hop=hop.stack()
hop.name="fa"
hop = hop.reset_index()
hop['hazard'] = 'drought'
hop.sample(3)

,iso3,rp,fa,hazard
113,KEN,25,0.008766,drought
3,AGO,25,0.018600,drought
164,OMN,100,0.501766,drought


In [36]:
hop['countryname'] = hop.iso3.replace(pd.read_csv("data/disasters/names.csv", index_col='iso3').country)

In [37]:
hop.sample(3)

,iso3,rp,fa,hazard,countryname
26,BRA,5,0.000015,drought,Brazil
233,VEN,100,0.006584,drought,Venezuela (Bolivarian Republic of)
223,TZA,100,0.011545,drought,"Tanzania, United Republic of"


In [44]:
hazards = pd.read_csv("data/disasters/fa_guessed_from_GAR_and_PAGER_GLOFRIS.csv")

In [45]:
hazards = hazards.append(hop,ignore_index=True)

In [49]:
hazards.hazard.unique()

array(['earthquake', 'flood', 'surge', 'tsunami', 'wind', 'floodglofris',
       'drought'], dtype=object)

In [51]:
hazards.to_csv("data/disasters/fa_guessed_from_GAR_and_PAGER_GLOFRIS.csv",index=False,encoding='utf8')

In [47]:
def correct_countrycode(countrycode):
    '''
    Corrects countrycodes in the database that don't correspond to official 3 letters codes.
    '''
    if countrycode=='TMP':
        countrycode='TLS'
    if countrycode=='ZAR':
        countrycode='COD'
    if countrycode=='ROM':
        countrycode='ROU'
    return countrycode

In [17]:
deltares_floods = pd.read_csv("data/disasters/deltares_floods_rp.csv", index_col="country")
deltares_floods.columns.name="rp"
hop=pd.DataFrame(deltares_floods.stack(), columns=["fa"]).reset_index()
hop['iso3'] = hop.country.replace(pd.read_csv("data/disasters/names.csv", index_col='country').iso3)
hop['hazard'] = 'floodglofris'

In [42]:
hop = hop.rename(columns={'country':'countryname'})

In [43]:
hop.head()

,countryname,rp,fa,iso3,hazard
0,Afghanistan,5,0.071899,AFG,floodglofris
1,Afghanistan,10,0.075261,AFG,floodglofris
2,Afghanistan,25,0.080745,AFG,floodglofris
3,Afghanistan,50,0.082926,AFG,floodglofris
4,Afghanistan,100,0.085365,AFG,floodglofris


In [9]:
hazards = pd.read_csv("data/disasters/fa_guessed_from_GAR_and_PAGER.csv")

In [46]:
hazards = hazards.append(hop,ignore_index=True).drop("Unnamed: 0",axis=1)

In [49]:
hazards.sample(10)

,countryname,fa,hazard,iso3,rp
3707,Slovenia,0.065906,flood,SVN,500
2532,Lithuania,0.000000,earthquake,LTU,1000
77,Argentina,0.007070,flood,ARG,20
273,Bangladesh,0.003133,wind,BGD,20
5391,Lesotho,0.022905,floodglofris,LSO,100
1550,France,0.000000,wind,FRA,250
4836,Canada,0.032609,floodglofris,CAN,10
3159,Niger,0.000000,surge,NER,100
1108,Cote d'Ivoire,0.000000,wind,CIV,100
727,Cameroon,0.000000,tsunami,CMR,1500


In [51]:
hazards.to_csv("data/disasters/fa_guessed_from_GAR_and_PAGER_GLOFRIS.csv",index=False,encoding='utf8')

In [18]:
df_original = pd.read_csv("data/disasters/df_original.csv")
#df_original['iso3'] = df_original.country.replace(pd.read_csv("data/disasters/names.csv", index_col='country').iso3)
#df_original.to_csv("data/disasters/df_original.csv")

In [ ]:
df_original

In [24]:
gar = pd.read_csv("data/disasters/fa_guessed_from_GAR_and_PAGER.csv")

In [25]:
gar.columns

Index(['countryname', 'hazard', 'rp', 'fa'], dtype='object')

In [26]:
gar['iso3'] = gar.countryname.replace(pd.read_csv("data/disasters/names.csv", index_col='country').iso3)

In [27]:
gar.to_csv("data/disasters/fa_guessed_from_GAR_and_PAGER.csv")

In [9]:
sub=gar.ix[gar.iso3=='MWI',:]

In [15]:
for (name,subsub) in sub.groupby('hazard'):
    print(name)
    a=subsub[['rp','fa']]

earthquake
flood
surge
tsunami
wind


In [31]:
codes = pd.read_csv('wbccodes2014.csv')
codes.rename(columns={'country':'ISO'},inplace=True)
codes['ISO'] = codes.ISO.apply(correct_countrycode)

In [32]:
gar.merge(codes.drop('Unnamed: 0',axis=1),on='country_name').to_csv("data/disasters/gar_for_model.csv")

In [33]:
aal_today = gar.merge(codes.drop('Unnamed: 0',axis=1),on='country_name')

In [35]:
aal_today.fillna(0)

,country_name,Income group,EXPOSED VALUE NATIONAL million US$,EARTHQUAKE million US$,EARTHQUAKE per th,TROPICAL CYCLONES million US$,TROPICAL CYCLONES per th,TSUNAMI million US$,TSUNAMI per th,FLOODS million US$,FLOODS per th,MULTI-HAZARD million US$,MULTI-HAZARD per th,ISO,wbregion,wbregionname,wbincome,wbincomename,wblending,wbhipc
0,Afghanistan,Low income,"60,187.90",146.81,2.44,0,0.00,0,0.00,74.52,1.24,221.33,3.68,AFG,SAS,South Asia,LIC,Low income,IDA,HIPC
1,Albania,Upper middle income,"40,459.70",46.73,1.15,0,0.00,0,0.00,27.74,0.69,74.47,1.84,ALB,ECA,Europe & Central Asia,UMI,Upper middle income,IBRD,0
2,Algeria,Upper middle income,"899,206.00",991.52,1.10,0,0.00,0,0.00,161.69,0.18,"1,153.21",1.28,DZA,MNA,Middle East & North Africa,UMI,Upper middle income,IBRD,0
3,American Samoa,Lower middle income,"1,930.49",0.4,0.21,14.29,7.40,0.01,0.01,0,0.00,14.7,7.61,ASM,EAP,East Asia & Pacific,UMI,Upper middle income,..,0
4,Andorra,N.D,"8,381.65",0.12,0.01,0,0.00,0,0.00,0,0.00,0.12,0.01,ADO,YHI,High Income,HIX,High income: nonOECD,..,0
5,Angola,Upper middle income,"176,183.00",4.6,0.03,0,0.00,0,0.00,159.95,0.91,164.55,0.93,AGO,SSA,Sub-Saharan Africa,UMI,Upper middle income,IDA,0
6,Antigua and Barbuda,High income: nonOECD,"6,257.29",30.93,4.94,238.29,38.08,0.06,0.01,0,0.00,269.28,43.03,ATG,YHI,High Income,HIX,High income: nonOECD,IBRD,0
7,Argentina,Upper middle income,"1,380,560.00","1,103.92",0.80,0,0.00,0,0.00,648.2,0.47,"1,752.12",1.27,ARG,LAC,Latin America & Caribbean,UMI,Upper middle income,IBRD,0
8,Armenia,Lower middle income,"22,895.20",45.24,1.98,0,0.00,0,0.00,17.68,0.77,62.92,2.75,ARM,ECA,Europe & Central Asia,LMI,Lower middle income,Blend,0
9,Aruba,High income: nonOECD,"8,909.38",16.14,1.81,39.35,4.42,0,0.00,0,0.00,55.49,6.23,ABW,YHI,High Income,HIX,High income: nonOECD,..,0


In [7]:
data1 = pd.read_csv("data/disasters/aal_today.csv",index_col='iso3')

In [131]:
data1.columns

Index(['country', 'exposed value million US$', 'earthquake (‰)',
       'cyclones (‰)', 'tsunami (‰)', 'floods (‰)', 'multi-hazard (‰)'],
      dtype='object')

In [15]:
df = pd.read_csv("data/disasters/df.csv")

In [133]:
df.columns

Index(['country', 'H', 'T_rebuild_K', 'T_rebuild_L', 'alpha', 'avg_prod_k',
       'axfin_p', 'axfin_r', 'axhealth', 'bashs', 'borrow_abi', 'cp', 'cr',
       'dC_destitution', 'dKpc', 'dKtot', 'dW_destitution', 'dW_no_transfers',
       'dW_noupscale', 'dWsurWprime', 'delta_W', 'destitution_exposure',
       'equivalent_cost', 'fa', 'fap', 'far', 'faref', 'finance_pre',
       'gdp_pc_pp', 'income_elast', 'individual_exposure',
       'institutional_exposure', 'iso3', 'macro_multiplier', 'ophe',
       'original_country', 'pe', 'peref', 'pi', 'plgp', 'pop', 'pov_head',
       'prepare_scaleup', 'protection', 'protectionref', 'pv', 'rating',
       'resilience', 'resilience_no_shock', 'resilience_no_shock_no_SP',
       'resilience_no_shock_no_uspcale', 'risk', 'risk_to_assets', 'rp',
       'share1', 'share1_ref', 'shew', 'sigma_p', 'sigma_r', 'social_p',
       'social_r', 'tot_p', 'tot_r', 'total_equivalent_cost',
       'total_equivalent_cost_no_destitution',
       'total_equivale

In [16]:
floods = df[['iso3','v','risk_to_assets','avg_prod_k']]

In [18]:
floods['floods_RI'] = floods.risk_to_assets/floods.avg_prod_k/floods.v

C:\Miniconda3\lib\site-packages\IPython\kernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [19]:
floods.to_csv("data/disasters/floods_RI.csv")

In [14]:
hist_drought = pd.read_excel("data/disasters/droughts_data.xlsx",sheetname='EUWATCH_ENSEMBLEMEAN_FUTURE')
hist_drought = hist_drought[['ISO','HIST_RP100']]
hist_drought.set_index('ISO').to_csv("data/disasters/hist_drought_deltares.csv")

In [119]:
coastal = pd.read_excel("data/disasters/costal_floods_data.xlsx")

In [124]:
coastal = coastal.merge(codes.rename(columns={'country_name':'Name'})[['ISO','Name']],on='Name')

In [125]:
pop_2010 = get_pop_data_from_ssp(ssp,5,2010)

In [127]:
coastal = coastal.merge(pop_2010,on='ISO')

In [128]:
coastal.columns

Index(['Name', 'Region', 'exposed_population_coastal_flooding_2010', 'ISO',
       'tot_pop_ssp5'],
      dtype='object')

In [129]:
coastal['share_2010'] = coastal.exposed_population_coastal_flooding_2010/coastal.tot_pop_ssp5

In [130]:
coastal[['share_2010','ISO']].set_index('ISO').to_csv('data/disasters/costal_floods_deltares.csv')